# Render Factory (Always Latest from GitHub)

Render-only notebook. It always syncs `/content/repo` to `origin/main` before rendering.

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
from pathlib import Path

ENABLE_DRIVE = True
REPO_NAME = 'wt-knight-text-animations'

if ENABLE_DRIVE:
    drive.mount('/content/drive')
    DRIVE_ROOT = Path('/content/drive/MyDrive/render_factory') / REPO_NAME
    for folder in ['cache', 'jobs', 'outputs']:
        (DRIVE_ROOT / folder).mkdir(parents=True, exist_ok=True)
    print('DRIVE_ROOT:', DRIVE_ROOT)
else:
    DRIVE_ROOT = None
    print('Drive disabled')

In [ ]:
import os
from getpass import getpass

GITHUB_REPO_URL = ''  # e.g. https://github.com/owner/repo.git
if not GITHUB_REPO_URL:
    GITHUB_REPO_URL = input('Enter GitHub repo URL: ').strip()

repo_url = GITHUB_REPO_URL
if 'github.com' in GITHUB_REPO_URL:
    is_private = input('Private repo? (y/N): ').strip().lower() == 'y'
    if is_private:
        token = getpass('GitHub token (not stored): ').strip()
        repo_url = GITHUB_REPO_URL.replace('https://', f'https://{token}@')

if os.path.exists('/content/repo/.git'):
    %cd /content/repo
    !git remote set-url origin "$repo_url"
    !git fetch origin
    !git checkout main
    !git reset --hard origin/main
    !git clean -fd
else:
    if os.path.exists('/content/repo'):
        !rm -rf /content/repo
    !git clone "$repo_url" /content/repo
    %cd /content/repo

print('Current commit:')
!git rev-parse HEAD

In [ ]:
!apt-get update -y
!apt-get install -y ffmpeg libnss3 libatk-bridge2.0-0 libgtk-3-0 libgbm1 libasound2

In [ ]:
import shutil
from pathlib import Path

CACHE_NODE_MODULES = DRIVE_ROOT / 'cache' / 'node_modules' if DRIVE_ROOT else None
target_nm = Path('/content/repo/node_modules')

%cd /content/repo
!node -v
!npm -v

if CACHE_NODE_MODULES and CACHE_NODE_MODULES.exists():
    print('Restoring node_modules cache from Drive...')
    if target_nm.exists():
        shutil.rmtree(target_nm)
    shutil.copytree(CACHE_NODE_MODULES, target_nm)
else:
    print('No cache found, running npm ci...')
    !npm ci
    if CACHE_NODE_MODULES:
        print('Saving node_modules cache to Drive...')
        if CACHE_NODE_MODULES.exists():
            shutil.rmtree(CACHE_NODE_MODULES)
        shutil.copytree(target_nm, CACHE_NODE_MODULES)

In [ ]:
from google.colab import files
from pathlib import Path
import json

JOB_PATH = ''  # Optional Drive path, e.g. /content/drive/MyDrive/render_factory/wt-knight-text-animations/jobs/job.json
USE_UPLOAD = JOB_PATH.strip() == ''

if USE_UPLOAD:
    uploaded = files.upload()
    if 'job.json' in uploaded:
        JOB_PATH = '/content/job.json'
    else:
        json_files = [f'/content/{k}' for k in uploaded.keys() if k.lower().endswith('.json')]
        if not json_files:
            raise RuntimeError('No job JSON uploaded. Upload job.json.')
        JOB_PATH = json_files[0]

print('JOB_PATH:', JOB_PATH)
job = json.loads(Path(JOB_PATH).read_text())
print('Job keys:', list(job.keys()))

In [ ]:
import json
from pathlib import Path
from datetime import datetime
import shutil

%cd /content/repo
!npm run render:job -- --job "$JOB_PATH"

job = json.loads(Path(JOB_PATH).read_text())
out_path = Path(job.get('outPath', '/content/out.mp4'))
if not out_path.exists():
    raise RuntimeError(f'Render failed, output missing: {out_path}')
print('Rendered:', out_path)

if DRIVE_ROOT:
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    drive_out = DRIVE_ROOT / 'outputs' / f'{out_path.stem}_{ts}{out_path.suffix}'
    shutil.copy2(out_path, drive_out)
    print('Copied to Drive:', drive_out)